In [66]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
eval_iters = 250
learning_rate = 3e-4
max_iters = 10000

cuda


In [68]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
          text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [70]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])


encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)

print(encoded_hello)
print(decoded_hello)


[61, 58, 65, 65, 68]
hello


In [72]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [74]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('Inputs:')
print(x)
print('Target:')
print(y)


Inputs:
tensor([[54, 75, 58,  1, 54, 67,  1, 62],
        [68, 76,  9,  3,  1, 72, 54, 62],
        [60, 65, 58, 73, 72, 24,  1, 49],
        [ 1, 57, 62, 57,  1, 67, 68, 73]], device='cuda:0')
Target:
tensor([[75, 58,  1, 54, 67,  1, 62, 57],
        [76,  9,  3,  1, 72, 54, 62, 57],
        [65, 58, 73, 72, 24,  1, 49, 68],
        [57, 62, 57,  1, 67, 68, 73,  0]], device='cuda:0')


In [76]:

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is ', context, ' target is', target)

when input is  tensor([80])  target is tensor(1)
when input is  tensor([80,  1])  target is tensor(1)
when input is  tensor([80,  1,  1])  target is tensor(28)
when input is  tensor([80,  1,  1, 28])  target is tensor(39)
when input is  tensor([80,  1,  1, 28, 39])  target is tensor(42)
when input is  tensor([80,  1,  1, 28, 39, 42])  target is tensor(39)
when input is  tensor([80,  1,  1, 28, 39, 42, 39])  target is tensor(44)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44])  target is tensor(32)


In [78]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model.forward(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [94]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply sigmoid to get probabilities
            probs = F.sigmoid(logits) 
            # print(probs)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=10)[0].tolist())
print(generated_chars)
print(model.parameters())


V;TiRTkb4c
<generator object Module.parameters at 0x00000142242F10E0>


In [82]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())
    

step: 0, train loss: 4.919, val loss: 4.931
step: 250, train loss: 4.839, val loss: 4.858
step: 500, train loss: 4.786, val loss: 4.765
step: 750, train loss: 4.726, val loss: 4.727
step: 1000, train loss: 4.651, val loss: 4.650
step: 1250, train loss: 4.617, val loss: 4.598
step: 1500, train loss: 4.546, val loss: 4.541
step: 1750, train loss: 4.472, val loss: 4.491
step: 2000, train loss: 4.447, val loss: 4.434
step: 2250, train loss: 4.395, val loss: 4.402
step: 2500, train loss: 4.334, val loss: 4.324
step: 2750, train loss: 4.268, val loss: 4.271
step: 3000, train loss: 4.231, val loss: 4.224
step: 3250, train loss: 4.157, val loss: 4.174
step: 3500, train loss: 4.119, val loss: 4.127
step: 3750, train loss: 4.072, val loss: 4.079
step: 4000, train loss: 4.028, val loss: 4.050
step: 4250, train loss: 3.987, val loss: 3.983
step: 4500, train loss: 3.934, val loss: 3.951
step: 4750, train loss: 3.908, val loss: 3.899
step: 5000, train loss: 3.829, val loss: 3.849
step: 5250, train l